In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import re
import time
import textwrap
from tqdm.notebook import tqdm
from pathlib2 import Path
import os
import math

import scipy
from scipy import stats
from scipy.optimize import minimize
from scipy.stats import chi2_contingency
from functools import partial

from numba import jit

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.pylab as pylab
import seaborn as sns
sns.set_style('whitegrid', {'legend.frameon':True})
sns.set_palette(sns.color_palette("Set1", 12))
#sns.set_context("paper")
fontsize = 12
params = {'legend.fontsize': fontsize,
  'figure.figsize': (18, 15),
  'axes.labelsize': fontsize,
  'axes.titlesize':fontsize,
  'axes.edgecolor':"0.3",
  'xtick.labelsize':fontsize,
  'ytick.labelsize':fontsize,
  'legend.fontsize':10,
  'font.size':fontsize,
  'font.family':'serif'}
pylab.rcParams.update(params)
plt.rc('axes', labelsize=fontsize) 
#plt.style.use('ggplot')
%matplotlib inline

In [4]:
import sys
sys.path.append('../')

from helpers import numba_config
from metric_store import save_metrics, save_metric, load_metrics, get_metric_names, load_metric
from corr_network import load_data, get_available_mask
from network_metrics import prepare_metric
from pipeline.pipeline import load_config
from g_test_for_metrics.g_test_for_metrics import get_metric_indicators, get_sign_for_metric

config_name = "pipeline.config"
config = load_config(config_name)

print(config.prefix_for_corr)

window_2d_delay_0d


In [5]:
signs_metric = {
    'probability_for_metrics/input_data/MSLP': False, 
    'probability_for_metrics/input_data/MSLP_preproc': False,
    'probability_for_metrics/network_metrics/LCC_w': True, 
    'probability_for_metrics/network_metrics/degree_w': False, 
    'probability_for_metrics/network_metrics/EVC_w': False,
    'probability_for_metrics/network_metrics/closeness_w': True,
    'probability_for_metrics/network_metrics/LCC_0.9': True,
    'probability_for_metrics/network_metrics/degree_0.9': False,
    'probability_for_metrics/network_metrics/EVC_0.9': False,
    'probability_for_metrics/network_metrics/closeness_0.9': False,
    'probability_for_metrics/network_metrics/LCC_0.95': True,
    'probability_for_metrics/network_metrics/degree_0.95': False,
    'probability_for_metrics/network_metrics/EVC_0.95': False,
    'probability_for_metrics/network_metrics/closeness_0.95': False,
}

In [6]:
def get_full_names_for_metric(metric_names):
    full_metric_names = list(get_metric_names(config, prefix='probability_for_metrics').keys())
    selected_metric_names = []
    for name in metric_names:
        for full_name in full_metric_names:
            if full_name.endswith(name) and full_name.find('diff_metrics') == -1:
                selected_metric_names += [full_name]
                break
    return selected_metric_names


def special_loading_of_metrics(config, selected_metric_names):
    metrics = []
    for metric_name in selected_metric_names:
        config.metrics_plot_options['metric_name'] = metric_name
        metric = load_metric(config, metric_name).astype('float32')
        metric = metric if signs_metric[metric_name] else 1 - metric
        metric = -np.log10(metric + 1e-10)
        metrics.append(metric)
        print(metric_name, metric.shape)
    return metrics


def compute_f1_score(fn, fp, tp):
    if (tp == 0) and (fp == 0) and (fn == 0):
        f1_score = 0
    else:
        f1_score = (2 * tp) / (2 * tp + fp + fn)
    return f1_score


def compute_balanced_accuracy(tn, fn, fp, tp):
    if (tp + fn == 0) and (tn + fp == 0):
        b_acc = 0
    else:
        b_acc = 0.5 * ((tp / (tp + fn)) + (tn / (tn + fp)))
    return b_acc


def compute_matthews_coefficient(tn, fn, fp, tp):
    if (tp + fp == 0) or (tp + fn == 0) or (tn + fp == 0) or (tn + fn == 0):
        mcc = 0
    else:
        denominator1 = math.sqrt(tp + fp) * math.sqrt(tp + fn)
        denominator2 = math.sqrt(tn + fp) * math.sqrt(tn + fn)
        mcc = ((tp / denominator1) * (tn / denominator2)) - ((fp / denominator1) * (fn / denominator2))
    return mcc


def compute_tpr_fpr_f1_bacc_mcc(contigency_table):
    tn, fn, fp, tp = contigency_table.ravel()
    tn, fn, fp, tp = map(float, [tn, fn, fp, tp])
    
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    f1 = compute_f1_score(fn, fp, tp)
    bacc = compute_balanced_accuracy(tn, fn, fp, tp)
    mcc = compute_matthews_coefficient(tn, fn, fp, tp)
    
    return tpr, fpr, f1, bacc, mcc


@jit(nopython = numba_config.nopython, nogil=numba_config.nogil, cache=numba_config.cache, error_model="numpy")
def estimate_prediction(predicted_events_1d, cyclone_events_1d, w):
    cyclone_events_1d_start = np.zeros_like(cyclone_events_1d)
    nt = len(predicted_events_1d)
    blocked = np.zeros_like(predicted_events_1d)
    for i in range(1, nt):
        if cyclone_events_1d[i] > 0:
            if cyclone_events_1d[i - 1] == 0:
                cyclone_events_1d_start[i] = 1
            else:
                blocked[i] = 1
    # Blocked is 1 if cyclone already started
    # Blocked is 1 if cyclone was predicted in this moment
    tn, fn, fp, tp = 0, 0, 0, 0
    for i in range(nt):
        if predicted_events_1d[i]:
            if not blocked[i]:
                if np.any(cyclone_events_1d_start[i:i + w]):
                    tp += 1
                else:
                    fp += 1
                blocked[i:i + w] = 1
        else:
            if not blocked[i]:
                if np.any(cyclone_events_1d_start[i:i + w]):
                    fn += 1
                else:
                    tn += 1
                
    tn //= w
    fn //= w
    
    contigency_table = np.array([[tn, fn], [fp, tp]])
    
    return contigency_table


def perform_g_test(contigency_table):
    if ((contigency_table[0, 0] == 0) and (contigency_table[0, 1] == 0)) or ((contigency_table[1, 0] == 0) and (contigency_table[1, 1] == 0)):
        I_stat = g_stat = 0
        p_val = np.nan
    else:
        g_stat, p_val, dof, expctd = chi2_contingency(contigency_table, lambda_="log-likelihood", correction=False)
        I_stat = g_stat / contigency_table.flatten().sum() / 2
    return g_stat, p_val, I_stat


def compute_weighted_sum(weights, metrics):
    weighted_metric = metrics[0] * weights[0]
    for i in range(1, len(metrics)):
        weighted_metric += metrics[i] * weights[i]
    return weighted_metric


def compute_contigency_table(weights, metrics, cyclone_events, window):
    metric_thr = 1
    weighted_metric = compute_weighted_sum(weights, metrics)
    predicted_events = (weighted_metric > metric_thr)
    predicted_events_1d = predicted_events.sum(axis=(0, 1))
    cyclone_events_1d = cyclone_events.sum(axis=(0, 1))
    contigency_table = estimate_prediction(predicted_events_1d, cyclone_events_1d, w=window)
    return contigency_table


def compute_quality_1(weights, metrics, cyclone_events, window):
    contigency_table = compute_contigency_table(weights, metrics, cyclone_events, window)
    g_stat, _, _ = perform_g_test(contigency_table)    
    return -g_stat

def get_tau_n_quality(contigency_table):
    (tn, fn), (fp, tp) = contigency_table
    quality = 1 - (fn / (fn + tp) + (fp + tp) / (fp + tn + tp + fn))
    return quality
    
def compute_quality_2(weights, metrics, cyclone_events, window):
    contigency_table = compute_contigency_table(weights, metrics, cyclone_events, window)
    quality = get_tau_n_quality(contigency_table)
    return -quality

compute_quality = {'g_stat': compute_quality_1, 'tau_n_quality': compute_quality_2}

print_initial = True
def optimization(metrics, cyclone_events, window, quality_opt_func, number_random_iter=40, max_minimize_iter=500):
    global print_initial
    compute_quality_all = partial(compute_quality[quality_opt_func], metrics=metrics, cyclone_events=cyclone_events, window=window)
    
    def show_progress_pbar(x, pbar):
        global print_initial

        f = compute_quality_all(x)

        if print_initial:
            print(f"initial quality = {-f:.2f}, initial x = {x}")
            print_initial = False

        pbar.set_description(f"current quality = {-f:.2f}, x = {x}")
        pbar.update(1)
    
    
    pbar_for_random_iter = tqdm(range(0, number_random_iter))
    
    optimal_g_stat = sys.maxsize # большое положительное число
    optimal_weights = []
    for ri in pbar_for_random_iter:
        pbar_for_random_iter.set_postfix({'random_iter #': ri+1})
        random_initial_weights = np.random.uniform(0.0, 0.5, len(metrics))
        if len(random_initial_weights) > 1:
            random_initial_weights /= np.sum(random_initial_weights)
        
        print_initial = True
        show_minimize_progress = partial(show_progress_pbar, pbar=tqdm(total=max_minimize_iter))
        result = minimize(compute_quality_all, random_initial_weights, bounds=[(0, 1)]*len(metrics), method='nelder-mead',
                          options={'return_all': True, 'maxiter': max_minimize_iter}, callback=show_minimize_progress)
        negative_g_stat = compute_quality_all(result.x)
        if negative_g_stat < optimal_g_stat:
            optimal_g_stat = negative_g_stat
            optimal_weights = result.x

    return optimal_weights


def calc_quality_metrics_for_optimal_combination(metrics, cyclone_events, optimal_weights, window):
    contigency_table = compute_contigency_table(optimal_weights, metrics, cyclone_events, window)
    g_stat, _, _ = perform_g_test(contigency_table)
    quality_tau_n = get_tau_n_quality(contigency_table)
    tpr, fpr, f1, bacc, mcc = compute_tpr_fpr_f1_bacc_mcc(contigency_table)
    return contigency_table, tpr, fpr, g_stat, f1, bacc, mcc, quality_tau_n

In [7]:
def iteration_by_metrics(cyclone_events, base_metric, metrics_list, suff, window, number_random_iter, quality_opt_func, autosave=False, path=''):
    result_dict = {}
    metrics_rating = {str(key): [] for key in metrics_list}
    ind = 0
    for sub_metrics_list in metrics_list:
        print('\n', sub_metrics_list)
        considered_metrics = [base_metric] + [f'{m}_{suff}' for m in sub_metrics_list]
        considered_metrics = get_full_names_for_metric(considered_metrics)
        print(considered_metrics)
        
        metrics = special_loading_of_metrics(config, considered_metrics)

        optimal_weights = optimization(metrics, cyclone_events, window, quality_opt_func, number_random_iter)
        contigency_table, tpr, fpr, g_stat, f1, bacc, mcc, quality_tau_n = \
            calc_quality_metrics_for_optimal_combination(metrics, cyclone_events, optimal_weights, window)
        result_dict[f'subset{ind}'] = [
            considered_metrics, 
            optimal_weights.tolist(), 
            contigency_table.tolist(), 
            tpr, fpr, g_stat, f1, bacc, mcc, quality_tau_n
        ]

        metrics_rating[str(sub_metrics_list)] = quality_tau_n #g_stat
        
        if autosave:
            f = open(f'{path}/temp_file.txt','w')
            f.write(str(result_dict))
            f.close()
            
        ind += 1
    
    return result_dict, metrics_rating


def save_optimization_results(result_dict, file_name):
    writer = ExcelWriter(file_name)
    
    for key in result_dict:
        result_df = pd.DataFrame(np.array(result_dict[key], dtype=object).reshape(1, len(result_dict[key])),
                                 columns=['metrics_subset', 'weights', 'CT', 'TPR', 'FPR', 'g_stat', 'f1', 'bacc', 'mcc', 'quality_tau_n'],
                                dtype=object)
        result_df.to_excel(writer, sheet_name=key, index=False)
    
    writer.save()


In [14]:
%%time

number_random_iter = 40

quality_opt_func = 'tau_n_quality' # 'g_stat' or 'tau_n_quality'
window = 1*8                       # 1*8 = 1 day; 2*8 = 2 day
base_metric = 'MSLP_preproc'       # MSLP or MSLP_preproc
suff = 'w'                         # 'w' or '0.9' or '0.95'
network_metrics_1 = [['LCC'], ['degree'], ['closeness'], ['EVC']]


cyclone_events = np.load('../cyclones_events.npz')['cyclone_events_2'] # track_size = 2 - для любого размера окна 
                                                                    # результаты одинаковые, т.к. важен только тот факт, 
                                                                    # что циклон где-то был в конкретный момент времени

global_path = config.event_predictions_options['work_dir'] / config.event_predictions_options['output_predictions_1d']

folder = f'{base_metric}_metrics_{suff}_{int(window/8)}m8w'
path = f'{global_path}/{folder}'
if not os.path.exists(path):
    os.makedirs(path, exist_ok=True)

    
# Only base_metric
file_name = f'{path}/ep1D_{base_metric}_{int(window/8)}m8w.xlsx'
result_dict, _ = iteration_by_metrics(cyclone_events, base_metric, [''], '', window, number_random_iter, quality_opt_func)
print(result_dict)
save_optimization_results(result_dict, file_name)

# MSLP + one metric
file_name = f'{path}/ep1D_{base_metric}_1metric_{suff}_{int(window/8)}m8w.xlsx'
result_dict, metrics_rating = iteration_by_metrics(cyclone_events, base_metric, network_metrics_1, suff, window, 
                                                   number_random_iter, quality_opt_func, True, path)
print(result_dict)
save_optimization_results(result_dict, file_name)

# MSLP + metrics
file_name = f'{path}/ep1D_{base_metric}_metrics_{suff}_{int(window/8)}m8w.xlsx'
metrics_priority = sorted(metrics_rating, key=metrics_rating.get, reverse=True)
metrics_priority = [eval(m)[0] for m in metrics_priority]
pd.DataFrame({'metrics_priority': metrics_priority}).to_csv(f'{file_name.split(".xlsx")[0]}_metrics_order.txt', index=False)
print('metrics_priority:', metrics_priority)
network_metrics_2 = [metrics_priority[0:2+i] for i in range(0, len(metrics_priority)-1)]
print(network_metrics_2)
result_dict, _ = iteration_by_metrics(cyclone_events, base_metric, network_metrics_2, suff, window, 
                                      number_random_iter, quality_opt_func, True, path)
print(result_dict)
save_optimization_results(result_dict, file_name)


 
[]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 0)

In [13]:
global_path

WindowsPath('Z:/Research/Climate/data/ERA5/ERA5_MSL_1982_2020_3h_0.75/event_detection_window_2d_delay_0d/predictions_1d')